<a href="https://colab.research.google.com/github/paddy-03/Voice-Recognition-of-a-Singer/blob/master/Voice_Recognition_(singer).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
os.chdir('./drive/My Drive')

In [0]:
baseDir = '/content/drive/My Drive/Specs/'
trainDir = baseDir + 'train'
testDir = baseDir + 'test'
valDir = baseDir + 'validation'

In [0]:
width = 250
height = 250
epochs = 20
batch_size = 10
num_classes = 8

In [0]:
import keras
from keras.applications import VGG16
from keras.preprocessing.image import ImageDataGenerator
from keras import layers
from keras import models
from keras.layers import Conv2D, MaxPool2D, Flatten,Dense, Dropout

In [0]:
conv_base = VGG16(weights='imagenet',
include_top=False,
input_shape=(250, 250, 3))
conv_base.trainable = False  #freezing all layers of the vgg16 model

In [0]:
#dense layers
from keras import models
from keras import layers
from keras import optimizers

model = models.Sequential()
model.add(layers.Dense(1024, activation='relu',input_dim=7*7*512))
model.add(Dropout(0.2))
model.add(layers.Dense(8, activation='softmax'))

In [10]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1024)              8389632   
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 8200      
Total params: 8,397,832
Trainable params: 8,397,832
Non-trainable params: 0
_________________________________________________________________


In [36]:
conv_base.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 250, 250, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 250, 250, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 250, 250, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 125, 125, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 125, 125, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 125, 125, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 62, 62, 128)       0         
__________

In [0]:
#Extracting the features and labels as numpy arrays
#predict function of conv_base is used to get the features of the image
datagen = ImageDataGenerator(rescale=1./255)
batch_size = 20
def extract_features(directory, sample_count):
  features = np.zeros(shape=(sample_count, 7, 7, 512))
  labels = np.zeros(shape=(sample_count,8))
  generator = datagen.flow_from_directory( directory,
  target_size=(250, 250),
  batch_size=batch_size,
  shuffle = False,                               
  class_mode='categorical')
  print(generator.class_indices)
  i = 0
  for inputs_batch, labels_batch in generator:
    features_batch = conv_base.predict(inputs_batch)
    features[i * batch_size : (i + 1) * batch_size] = features_batch
    labels[i * batch_size : (i + 1) * batch_size,:] = labels_batch
    i += 1
    if i * batch_size >= sample_count:
        break
  return features, labels

In [39]:
train_features, train_labels = extract_features(trainDir, 1763)
validation_features, validation_labels = extract_features(valDir, 384)
test_features, test_labels = extract_features(testDir, 409)

Found 1763 images belonging to 8 classes.
{'Arijit Singh': 0, 'Armaan Malik': 1, 'Atif Aslam': 2, 'Honey Singh': 3, 'Raahat Fateh Ali Khan': 4, 'Shreya Ghoshal': 5, 'Sonu Nigam': 6, 'Sunidhi Chauhan': 7}
Found 384 images belonging to 8 classes.
{'Arijit Singh': 0, 'Armaan Malik': 1, 'Atif Aslam': 2, 'Honey Singh': 3, 'Raahat Fateh Ali Khan': 4, 'Shreya Ghoshal': 5, 'Sonu Nigam': 6, 'Sunidhi Chauhan': 7}
Found 409 images belonging to 8 classes.
{'Arijit Singh': 0, 'Armaan Malik': 1, 'Atif Aslam': 2, 'Honey Singh': 3, 'Raahat Fateh Ali Khan': 4, 'Shreya Ghoshal': 5, 'Sonu Nigam': 6, 'Sunidhi Chauhan': 7}


In [0]:
train_features = np.reshape(train_features, (1763, 7 * 7 * 512))
validation_features = np.reshape(validation_features, (384, 7 * 7 * 512))
test_features = np.reshape(test_features, (409, 7 * 7 * 512))

In [0]:
from keras import optimizers
model.compile(loss='categorical_crossentropy',optimizer=optimizers.RMSprop(lr = 1e-4),metrics=['acc'])

In [44]:
history = model.fit(train_features, train_labels,
epochs=100,
batch_size=20,
validation_data=(validation_features, validation_labels))

Train on 1763 samples, validate on 384 samples
Epoch 1/100
1763/1763 [==============================] - 3s 2ms/step - loss: 2.1735 - acc: 0.1923 - val_loss: 2.1604 - val_acc: 0.1823
Epoch 2/100
1763/1763 [==============================] - 3s 2ms/step - loss: 1.9454 - acc: 0.2479 - val_loss: 2.1300 - val_acc: 0.2656
Epoch 3/100
1763/1763 [==============================] - 3s 2ms/step - loss: 1.8102 - acc: 0.3210 - val_loss: 2.4979 - val_acc: 0.1302
Epoch 4/100
1763/1763 [==============================] - 3s 2ms/step - loss: 1.7404 - acc: 0.3454 - val_loss: 2.0658 - val_acc: 0.2812
Epoch 5/100
1763/1763 [==============================] - 3s 2ms/step - loss: 1.6378 - acc: 0.3948 - val_loss: 2.0691 - val_acc: 0.2292
Epoch 6/100
1763/1763 [==============================] - 3s 2ms/step - loss: 1.5956 - acc: 0.4124 - val_loss: 2.7971 - val_acc: 0.1276
Epoch 7/100
1763/1763 [==============================] - 3s 2ms/step - loss: 1.5721 - acc: 0.4243 - val_loss: 2.1573 - val_acc: 0.2005
Epoch 8/

In [45]:
model.evaluate(test_features,test_labels, batch_size = 20)

409/409 [==============================] - 0s 307us/step


[2.1987171322471646, 0.4523227398800675]

In [0]:
test_predictions = model.predict(test_features,batch_size=20)

In [47]:
test_predictions.shape

(409, 8)

In [0]:
y_pred = []
for i in test_predictions:
  max = -1
  pos=-1
  for j in range(8):
    if i[j] > max:
      max=i[j]
      pos = j
  y_pred.append(pos)

In [0]:
y_test = []
for i in test_labels:
  max = -1
  pos=-1
  for j in range(8):
    if i[j] > max:
      max=i[j]
      pos = j
  y_test.append(pos)

In [0]:
y_train = []
for i in train_labels:
  max = -1
  pos=-1
  for j in range(8):
    if i[j] > max:
      max=i[j]
      pos = j
  y_train.append(pos)

In [0]:
y_pred = np.array(y_pred)
y_test = np.array(y_test)
y_train = np.array(y_train)

In [0]:
from sklearn.metrics import confusion_matrix

In [53]:
print(confusion_matrix(y_test,y_pred))

[[ 9  7  6  3  4  1  1 17]
 [ 2 30 11  2  1  3  2  4]
 [ 2  5 21  0  5  1  0  6]
 [ 1  0  4 21 10  0  0 12]
 [ 3  3  6  0 36  3  1  4]
 [ 5  2  7  0  3 25  0 11]
 [ 4  2  6  0 14  1 13 16]
 [ 4  0  2  0 13  4  0 30]]


In [0]:
'Arijit Singh': 0, 'Armaan Malik': 1, 'Atif Aslam': 2, 'Honey Singh': 3, 'Raahat Fateh Ali Khan': 4, 'Shreya Ghoshal': 5, 'Sonu Nigam': 6, 'Sunidhi Chauhan': 7}

In [0]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier

In [57]:
clf = XGBClassifier(n_threads=-1,n_estimators=50)
clf.fit(train_features,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=50,
       n_jobs=1, n_threads=-1, nthread=None, objective='multi:softprob',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)

In [58]:
clf.score(test_features,y_test)

0.4254278728606357